In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Unzip MOT17 into the current working directory (root /content)
!unzip -q /content/drive/MyDrive/MOT17.zip -d /content
# Move MOT17 folder into the repo so paths match README
!mkdir -p /content/deepSORT_MOT17
!rsync -a /content/MOT17/ /content/deepSORT_MOT17/MOT17/

Streaming output truncated to the last 5000 lines.
  inflating: MOT17/train/MOT17-04-DPM/img1/000455.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000333.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/001039.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000327.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000441.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000496.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000482.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000509.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000521.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000247.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000253.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000535.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000284.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000290.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000079.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000045.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000723.jpg

In [3]:
!git clone https://github.com/Ulyssesllc/deepSORT_MOT17.git

Cloning into 'deepSORT_MOT17'...
remote: Enumerating objects: 11482, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11482 (delta 7), reused 12 (delta 6), pack-reused 11465 (from 1)
Receiving objects: 100% (11482/11482), 1.82 GiB | 41.00 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [ ]:
import os

os.chdir('deepSORT_MOT17')
!pip install -r requirements.txt
# Ensure TF packages available on Colab for feature extraction
!pip install -q tensorflow tf-keras tf-slim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 8.6 MB/s eta 0:00:00


In [ ]:
# Ensure the MARS ReID model exists at expected path
import pathlib
model_dst = pathlib.Path('resources/networks/mars-small128.pb')
model_dst.parent.mkdir(parents=True, exist_ok=True)
# If you have it on Drive, copy it; else download from your own link
# Example (adjust the source path accordingly):
# !cp /content/drive/MyDrive/models/mars-small128.pb resources/networks/mars-small128.pb
print('Model exists:', model_dst.exists(), '->', model_dst)

In [ ]:
# Optional: validate after features are generated. Skip now to avoid failing before creation.
# !python validate_paths.py \
#   --sequence_dir MOT17/train/MOT17-04-FRCNN \
#   --detection_file detections/MOT17-04-FRCNN.npy
print('Skipping validation pre-generation. Will validate after features are created.')

[FAIL] Không tìm thấy sequence_dir: MOT17/train/MOT17-04-FRCNN
[FAIL] Không tìm thấy detection_file: detections/MOT17-04-FRCNN.npy

Gợi ý:
- Đặt dữ liệu theo cấu trúc: MOT17/train/<sequence>/{img1/, det/det.txt, (gt/gt.txt)}
- Tạo đặc trưng bằng: python tools/generate_detections.py --mot_dir MOT17/train --output_dir detections


In [ ]:
!python tools/generate_detections.py \
  --model /content/deepSORT_MOT17/resources/networks/mars-small128.pb \
  --mot_dir MOT17/train \
  --output_dir detections

2025-08-28 04:25:02.308711: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756355102.604663    1612 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756355102.708651    1612 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756355103.300169    1612 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756355103.300213    1612 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756355103.300222    1612 computation_placer.cc:177] computation placer alr

In [ ]:
# Also generate features for the test split (no gt there)
!python tools/generate_detections.py \
  --model /content/deepSORT_MOT17/resources/networks/mars-small128.pb \
  --mot_dir MOT17/test \
  --output_dir detections

In [ ]:
# Now that features exist, validate one sequence path and its .npy
!python validate_paths.py \
  --sequence_dir MOT17/train/MOT17-04-FRCNN \
  --detection_file detections/MOT17-04-FRCNN.npy

In [7]:
!python evaluate_motchallenge.py \
  --mot_dir MOT17/train \
  --detection_dir detections \
  --output_dir results \
  --min_confidence 0.3 \
  --nms_max_overlap 1.0 \
  --max_cosine_distance 0.2 \
  --nn_budget 100

Traceback (most recent call last):
  File "/content/deepSORT_MOT17/evaluate_motchallenge.py", line 71, in <module>
    for d in sorted(os.listdir(args.mot_dir))
                    ^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'MOT17/train'


In [ ]:
# Evaluate metrics on train split only (test split has no GT)
!python evaluate_metrics.py \
  --mot_dir MOT17/train \
  --result_dir results \
  --output_csv mot17_train_metrics.csv \
  --min_iou 0.5

In [ ]:
!python deep_sort_app.py \
  --sequence_dir MOT17/train/MOT17-04-FRCNN \
  --detection_file detections/MOT17-04-FRCNN.npy \
  --output_file results/MOT17-04-FRCNN.txt \
  --min_confidence 0.3 \
  --nms_max_overlap 1.0 \
  --max_cosine_distance 0.2 \
  --nn_budget 100 \
  --display False

Traceback (most recent call last):
  File "/content/deepSORT_MOT17/deep_sort_app.py", line 332, in <module>
    run(
  File "/content/deepSORT_MOT17/deep_sort_app.py", line 200, in run
    seq_info = gather_sequence_info(sequence_dir, detection_file)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/deepSORT_MOT17/deep_sort_app.py", line 45, in gather_sequence_info
    raise FileNotFoundError(
FileNotFoundError: Không tìm thấy sequence_dir: MOT17/train/MOT17-04-FRCNN. Hãy đặt dữ liệu theo cấu trúc MOT17, ví dụ 'MOT17/train/MOT17-04-FRCNN'.


In [ ]:
# Visualize results: on Colab, this will render frames to a window-less viewer; use video export + inline preview below for display
!python show_results.py \
  --sequence_dir MOT17/train/MOT17-04-FRCNN \
  --result_file results/MOT17-04-FRCNN.txt \
  --detection_file detections/MOT17-04-FRCNN.npy \
  --update_ms 20

Traceback (most recent call last):
  File "/content/deepSORT_MOT17/show_results.py", line 128, in <module>
    run(
  File "/content/deepSORT_MOT17/show_results.py", line 43, in run
    seq_info = deep_sort_app.gather_sequence_info(sequence_dir, detection_file)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/deepSORT_MOT17/deep_sort_app.py", line 45, in gather_sequence_info
    raise FileNotFoundError(
FileNotFoundError: Không tìm thấy sequence_dir: MOT17/train/MOT17-04-FRCNN. Hãy đặt dữ liệu theo cấu trúc MOT17, ví dụ 'MOT17/train/MOT17-04-FRCNN'.


In [10]:
!python generate_videos.py \
  --mot_dir MOT17/train \
  --result_dir results \
  --output_dir videos \
  --update_ms 20 \
  --convert_h264

In [ ]:
# Inline preview the generated video on Colab
from IPython.display import HTML
from base64 import b64encode
video_path = 'videos/MOT17-04-FRCNN.mp4'
if not os.path.exists(video_path):
  # Fallback to a likely name; list videos
  import glob
  vids = glob.glob('videos/*.mp4')
  print('Videos found:', vids)
  if vids:
    video_path = vids[0]
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [11]:
!python evaluate_metrics.py \
  --mot_dir MOT17/train \
  --result_dir results \
  --output_csv mot17_train_metrics.csv \
  --min_iou 0.5

Traceback (most recent call last):
  File "/content/deepSORT_MOT17/evaluate_metrics.py", line 117, in <module>
    main()
  File "/content/deepSORT_MOT17/evaluate_metrics.py", line 106, in main
    summary, str_summary = evaluate_split(
                           ^^^^^^^^^^^^^^^
  File "/content/deepSORT_MOT17/evaluate_metrics.py", line 32, in evaluate_split
    d for d in os.listdir(mot_dir) if os.path.isdir(os.path.join(mot_dir, d))
               ^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'MOT17/train'
